In [1]:
!git clone https://github.com/zhangbo2008/opensource_tts_chinese_and_english

Cloning into 'opensource_tts_chinese_and_english'...
remote: Enumerating objects: 2640, done.
remote: Total 2640 (delta 0), reused 0 (delta 0), pack-reused 2640
Receiving objects: 100% (2640/2640), 54.93 MiB | 27.03 MiB/s, done.
Resolving deltas: 100% (971/971), done.


In [2]:
%cd opensource_tts_chinese_and_english

/kaggle/working/opensource_tts_chinese_and_english


In [3]:
!mv  transformers transformers9999

In [4]:
%ls

11.wav                   666/            chinese_tts2.py
2.mp3                    8888.py         detr_model_debug.py
2021-06-16-16-29-56.wav  ChineseASR.py   english_tts.py
2021-07-01-17-42-26.wav  EnglishASR.py   readme.md
2222222.py               TensorFlowTTS/  tensorflow_tts/
22English.wav            __init__.py     test.tsv
3333333.py               audio.wav       transformers9999/
444.py                   chinese.wav     vit_model_debug.py
444的语音文件.flac       chinese_asr.py  刷时间.py
65555.py                 chinese_tts.py  语音识别中文字典


In [5]:
!pip install tensorflow==2.12.0


In [6]:
!pip install jamo inflect pypinyin  tensorflow_addons  g2p_en


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 37.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 81.2 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 14.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for distance: filename=Distance-0.1.3-py3-none-any.whl size=16258 sha256=4820b36ff8d42b14567aa033d7378d573cd4e55eb055ac26724d59fb063ff982
  Stored in directory: /root/.cache/pip/wheels/e8/bb/de/f71bf63559ea9a921059a5405806f7ff6ed612a9231c4a9309
Successfully built distance


In [7]:
!pip install transformers

In [12]:
!pip install librosa==0.10.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.7/253.7 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: librosa
    Found existing installation: librosa 0.9.2
    Uninstalling librosa-0.9.2:
      Successfully uninstalled librosa-0.9.2


In [17]:
import librosa
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from datasets import load_dataset  # pip 一下 datasets  , soundfile
import soundfile as sf

processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")


def map_to_array(batch):
    speech, _ = sf.read(batch["file"])

    batch["speech"] = speech  # 把文件放到speech的value里面
    return batch
if 0:
    ds = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split="validation")
    ds = ds.map(map_to_array)  # map : Apply a function to all the elements in the table (individually or in batches)
    # 我们只玩句子里面第一句.
    input_values = processor(ds["speech"][0], return_tensors="pt").input_values  # Batch size 1
    logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)

    transcription = processor.decode(predicted_ids[0])

     # compute loss
    target_transcription = "A MAN SAID TO THE UNIVERSE SIR I EXIST"

     # wrap processor as target processor to encode labels
    with processor.as_target_processor():
      labels = processor(target_transcription, return_tensors="pt").input_ids       # 把答案也进行编码,跟语音编码是一样的.上面用的特征提取器, 下面用的nlp编码.
    # 编码之后也是38这个长度.
    loss = model(input_values, labels=labels).loss
    print(loss)


#{'<pad>': 0, '<s>': 1, '</s>': 2, '<unk>': 3, '|': 4, 'E': 5, 'T': 6, 'A': 7, 'O': 8, 'N': 9, 'I': 10, 'H': 11, 'S': 12, 'R': 13, 'D': 14, 'L': 15, 'U': 16, 'M': 17, 'W': 18, 'C': 19, 'F': 20, 'G': 21, 'Y': 22, 'P': 23, 'B': 24, 'V': 25, 'K': 26, "'": 27, 'X': 28, 'J': 29, 'Q': 30, 'Z': 31} 编码字典是这个.



# 下面在自己的数据集上使用.

from transformers import Wav2Vec2Tokenizer, Wav2Vec2ForCTC
from datasets import load_dataset  # 这2个库包都pip一下就行.
import soundfile as sf
import torch
from datasets import load_dataset

# load model and tokenizer
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

# name='2.mp3'
name='11.wav'
name='22English.wav'
if ".wav" in name:
    src_sig,sr = sf.read(name)  #name是要 输入的wav 返回 src_sig:音频数据  sr:原采样频率
    dst_sig = librosa.resample(src_sig,orig_sr=sr,target_sr=16000)  #resample 入参三个 音频数据 原采样频率
# if ".mp3" in name :
#     from pydub import AudioSegment
#
#     sound = AudioSegment.from_file(name, format='MP3')
#     MP3_File = AudioSegment.from_mp3(file=name)
#     MP3_File.export('tmp.wav', format="wav")
#
#     src_sig,sr = sf.read('tmp.wav')  #name是要 输入的wav 返回 src_sig:音频数据  sr:原采样频率
#     dst_sig = librosa.resample(src_sig,sr,16000)  #resample 入参三个 音频数据 原采样频率

# 基本啥也没干的tokenizer
input_values = tokenizer(dst_sig, return_tensors="pt", padding="longest").input_values  # Batch size 1

# retrieve logits
logits = model(input_values).logits

# take argmax and decode
predicted_ids = torch.argmax(logits, dim=-1)
transcription = tokenizer.batch_decode(predicted_ids) # 下面看这个解码算法, 应该是vitebi.....其实不是,只是简单的拼接......因为都没给概率......
print(transcription)


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


["THIS IS A DEMOT TO SHOW HOW TO USE A MODEL TO GENERATE ROLE' SPECTROGRAM FROM RAW TEXT"]


In [ ]:
print('下面是我们的音频数据我们可以看一下上面生成的文本是不是对的')

In [24]:
Audio(name, autoplay=True)